In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf 
from tensorflow import keras
from keras import layers 
from keras import callbacks
from sklearn.compose import make_column_transformer 
from sklearn.model_selection import ShuffleSplit, train_test_split 
from sklearn.utils import shuffle

In [2]:
LineData = pd.read_csv("DataLine3.csv")
LineData.head()

,Distance,LG,LL,LLG,LLL,Ia,Ib,Ic,Va,Vb,Vc
0,30,0,0,1,0,701.870,129.640,194.240,6202.7,2191.0,1340.9
1,30,0,0,0,1,44.187,44.187,44.187,1094.5,1094.5,1094.5
2,20,0,1,0,0,701.020,298.850,402.230,6149.4,3439.7,2923.6
3,50,0,1,0,0,701.030,490.810,243.670,6149.4,4893.0,2344.5
4,50,0,0,1,0,698.930,140.510,194.590,6181.0,2271.4,1428.1


In [3]:
# Shuffling
LineData_suffled = shuffle(LineData, random_state = 2)
LineData_suffled.head()

,Distance,LG,LL,LLG,LLL,Ia,Ib,Ic,Va,Vb,Vc
331,20,0,1,0,0,701.02,687.12,624.78,6149.4,6126.8,5613.4
84,50,0,0,0,1,190.71,190.71,190.71,2649.7,2649.7,2649.7
436,35,0,0,0,1,1914.20,1914.20,1914.20,30042.0,30039.0,30042.0
394,20,1,0,0,0,604.77,701.20,703.69,5504.8,6162.1,6187.3
161,25,0,1,0,0,723.39,2180.30,2085.70,36365.0,29542.0,30124.0


In [4]:
features_col = ["Ia", "Ib", "Ic", "Va", "Vb", "Vc"] 
target_col = ["Distance"]
X = LineData_suffled[features_col] 
Y = LineData_suffled[target_col] 

In [5]:
scalling = StandardScaler()
X_scalled = scalling.fit_transform(X, Y) 
x_train, x_val, y_train, y_val  = train_test_split(X_scalled, Y, test_size=0.15, random_state=10) 

In [7]:
Dis_model = keras.Sequential([
    layers.Dense(60, activation='relu', input_shape=[6]), 
    layers.BatchNormalization(),
    layers.Dense(100, activation='tanh'), 
    layers.BatchNormalization(),
    layers.Dense(80, activation='relu'),
    layers.BatchNormalization(),  
    layers.Dense(80, activation='relu'), 
    layers.Dense(1) 
])


Dis_model.compile(
    optimizer = 'adam',
    loss = 'mae',
    metrics = ['mean_absolute_error'],
)

path = "camera ready models/model_{epoch:02d}_{val_mean_absolute_error:.2f}.hdf5"
checkpoint = callbacks.ModelCheckpoint(
    filepath = path,
    monitor = 'val_mean_absolute_error',
    verbose = 1,
    save_best_only=True,
    mode='min'
)

In [8]:
History = pd.DataFrame({}) 

In [24]:
history = Dis_model.fit(
    x_train, y_train,
    validation_data = (x_val, y_val),
    batch_size = 120,  
    epochs = 1000,
    callbacks=[checkpoint] 
)

Epoch 1/1000
1/4 [======>.......................] - ETA: 0s - loss: 3.5216 - mean_absolute_error: 3.5216
Epoch 1: val_mean_absolute_error did not improve from 4.84491
4/4 [==============================] - 0s 33ms/step - loss: 3.7739 - mean_absolute_error: 3.7739 - val_loss: 8.5845 - val_mean_absolute_error: 8.5845
Epoch 2/1000
1/4 [======>.......................] - ETA: 0s - loss: 2.5889 - mean_absolute_error: 2.5889
Epoch 2: val_mean_absolute_error did not improve from 4.84491
4/4 [==============================] - 0s 14ms/step - loss: 3.7696 - mean_absolute_error: 3.7696 - val_loss: 9.7779 - val_mean_absolute_error: 9.7779
Epoch 3/1000
1/4 [======>.......................] - ETA: 0s - loss: 4.2078 - mean_absolute_error: 4.2078
Epoch 3: val_mean_absolute_error did not improve from 4.84491
4/4 [==============================] - 0s 15ms/step - loss: 4.0385 - mean_absolute_error: 4.0385 - val_loss: 15.9770 - val_mean_absolute_error: 15.9770
Epoch 4/1000
1/4 [======>......................

In [25]:
history_df = pd.DataFrame(history.history)

In [26]:
History = pd.concat([History, history_df]) 

In [26]:
History.to_csv('Line 03 Data.csv') 

In [27]:
Best_model = keras.models.load_model("distance_saved_models/model_727_4.11.hdf5") 
y_pred = Best_model.predict(x_val)
y_val_series = y_val.squeeze()
y_pred_series = y_pred.squeeze()
comparison = pd.DataFrame({
    "True": np.array(y_val_series), 
    "Predicted": np.array(y_pred_series)  
})

3/3 [==============================] - 0s 2ms/step


In [28]:
comparison.to_csv("Line 03 train val comparison.csv")